In [1]:
import json
import os
import qumulo
from qumulo.rest_client import RestClient

In [2]:
API_HOSTNAME = '10.120.0.34'
API_USERNAME = 'admin'
API_PASSWORD = 'Admin123'

In [3]:
def cluster_login(api_hostname, api_username, api_password):
    """
    Log into cluster and return rest client object.
    """
    print('Logging into cluster...')
    rest_client = RestClient(api_hostname, 8000)
    rest_client.login(api_username, api_password)

    return rest_client

In [4]:
rest_client = cluster_login(API_HOSTNAME, API_USERNAME, API_PASSWORD)

Logging into cluster...


In [5]:
# rest_client.ad.list_ad()

In [6]:
# rest_client.cluster.list_nodes()

In [7]:
rest_client.cluster.list_nodes()[0]['node_status']

'online'

In [8]:
rest_client.node_state.get_node_state()

{'node_id': 1,
 'state': 'ACTIVE',
 'cluster_id': 'cf83e828-7ef7-4368-a75b-3b972d10f2c6'}

In [9]:
# rest_client.cluster.get_cluster_slots_status()

In [10]:
multilinestring = """
this
that
then
your mom
"""

In [11]:
multilinestring

'\nthis\nthat\nthen\nyour mom\n'

In [12]:
print(multilinestring)


this
that
then
your mom



In [13]:
# rest_client.cluster.get_cluster_slots_status()

In [14]:
os.getcwd()

'/Users/rthompson/Documents/Dev/Python/cluster-event-alerts'

In [15]:
# if 'cluster_state.json' in os.listdir():
#     os.rename('cluster_state.json','cluster_state_previous.json')
#     PREVIOUS_EXISTED = True

with open('cluster_state.json', 'w') as f:
    for data in rest_client.cluster.list_nodes():
        f.writelines(data)

In [16]:
# for item in rest_client.cluster.list_nodes():
#     print(item)

In [17]:
# rest_client.cluster.list_nodes()[:2]

In [18]:
rest_client.cluster.list_nodes()[0]

{'id': 1,
 'node_status': 'online',
 'node_name': 'CoffeeTime-1',
 'uuid': '10a1c7aa-fb99-48a1-8dc3-b34b96777742',
 'label': '00:50:56:bf:68:82',
 'model_number': 'QVIRT',
 'serial_number': 'QVIRT',
 'mac_address': '00:50:56:bf:68:82'}

In [19]:
node_relevant_fields = ['id','node_status','node_name','uuid','model_number','serial_number']
drive_relevant_fields = ['id','node_id','slot','state','slot_type','disk_type','disk_model','disk_serial_number','capacity']

In [20]:
newlist = []
for entry in range(len(rest_client.cluster.list_nodes())):
    newdict = {}
    for k,v in rest_client.cluster.list_nodes()[entry].items():
        if k in node_relevant_fields:
            newdict[k] = v
    newlist.append(newdict)

In [21]:
for entry in range(len(rest_client.cluster.list_nodes())):
    print(entry)

0
1
2
3


In [22]:
rest_client.cluster.list_nodes()[1].items()

dict_items([('id', 2), ('node_status', 'online'), ('node_name', 'CoffeeTime-2'), ('uuid', 'cbdea0e3-1659-48af-b15b-e97dbbeefd04'), ('label', '00:50:56:bf:f1:57'), ('model_number', 'QVIRT'), ('serial_number', 'QVIRT'), ('mac_address', '00:50:56:bf:f1:57')])

In [45]:
def get_qq_version(rest_client):
    """
    Query API for Qumulo Core version.
    """

    qq_version = rest_client.version.version()['revision_id']

    return qq_version

In [56]:
def get_cluster_time(rest_client):
    """
    Get cluster time and return as cluster_time.
    """
    
    cluster_time = rest_client.time_config.get_time_status()['time']
    
    return cluster_time

# cluster_time = get_cluster_time(rest_client)

In [23]:
def retrieve_status_of_cluster_nodes(rest_client):
    """
    Accept rest_client object to query via API call to retrieve info/status for
    nodes. Parse through information and record relevant information. Return
    dict object to later dump as json.
    """
    node_relevant_fields = [
        'id',
        'node_status',
        'node_name',
        'uuid',
        'model_number',
        'serial_number',
    ]

    temp_list = []
    for num in range(len(rest_client.cluster.list_nodes())):
        new_dict = {}
        for k,v in rest_client.cluster.list_nodes()[num].items():
            if k in node_relevant_fields:
                new_dict[k] = v
        temp_list.append(new_dict)
    
    status_of_nodes = {}
    status_of_nodes["nodes"] = temp_list
    
    return status_of_nodes

In [24]:
def retrieve_status_of_cluster_drives(rest_client):
    """
    Accept rest_client object to query via API call to retrieve info/status for
    drives. Parse through information and record relevant information. Return
    dict object to later dump as json.
    """
    drive_relevant_fields = [
        'id',
        'node_id',
        'slot',
        'state',
        'slot_type',
        'disk_type',
        'disk_model',
        'disk_serial_number',
        'capacity',
    ]

    temp_list = []
    for num in range(len(rest_client.cluster.get_cluster_slots_status())):
        new_dict = {}
        for k,v in rest_client.cluster.get_cluster_slots_status()[num].items():
            if k in drive_relevant_fields:
                new_dict[k] = v
        temp_list.append(new_dict)

    status_of_drives = {}
    status_of_drives["drives"] = temp_list
    
    return status_of_drives

In [25]:
status_of_nodes = retrieve_status_of_cluster_nodes(rest_client)
# status_of_nodes

In [26]:
status_of_drives = retrieve_status_of_cluster_drives(rest_client)
# status_of_drives

In [27]:
def combine_statuses_formatting(status_of_nodes, status_of_drives):
    """
    In order to adhere to proper json formatting, this func will combine the
    two status_of_nodes and status_of_drives dictionary objects into one
    single dictionary object.
    """
    status_of_nodes['drives'] = status_of_drives['drives']    
    cluster_status = status_of_nodes
    
    return cluster_status

In [28]:
cluster_status = combine_statuses_formatting(status_of_nodes, status_of_drives)

In [29]:
def check_for_previous_state(cluster_status):
    """
    If cluster_state.json exists, rename it to cluster_state_previous.json.
    Regardless of this, then create cluster_state.json and write node + drive
    statuses to file. Return variable for previous_existed.
    """
    if 'cluster_state.json' in os.listdir():
        os.rename('cluster_state.json','cluster_state_previous.json')
        previous_existed = True
    else:
        previous_existed = False
    with open('cluster_state.json', 'w') as f:
        json.dump(cluster_status, f, indent=4)
        # f.write('\n')
        # json.dump(status_of_drives, f, indent=4)

    return previous_existed

In [30]:
def compare_states():
    """
    If a previous state file existed, then compare new cluster state to previous
    cluster state and check for any changes, IE nodes offline or drive issues.
    Once the comparison is complete, remove the previous state file and rename
    cluster_state.json to cluster_state_previous.json.
    """
    
    # XXX: Might not need this
    changes = False
    
    # XXX: add logic here to compare the two files and look for CHANGES
    if previous_existed:
        pass  
    else:
        pass
    
    return changes

In [31]:
previous_existed = check_for_previous_state(cluster_status)

In [33]:
def check_for_unhealthy_objects():
    """ 
    Scan the cluster_state.json file to determine whether or not there are 
    unhealthy objects.
    """
    
    healthy = True
    with open('cluster_state_TEST.json') as f:
        data = json.load(f)
        alert_data = {}
        counter = 1
    
    for dictobj in data['nodes']:
        for k,v in dictobj.items():
            if k == 'node_status':
                if v != 'online':
                    print('ALERT!! UNHEALTHY NODE FOUND.')
                    alert_data[f'Event {counter}'] = dictobj
                    counter += 1
                    healthy = False

                    
    for dictobj in data['drives']:
        for k,v in dictobj.items():
            if k == 'state':
                if v != 'healthy':
                    print('ALERT!! UNHEALTHY DRIVE FOUND.')                    
                    alert_data[f'Event {counter}'] = dictobj   
                    counter += 1
                    healthy = False
        
    print(alert_data)

In [36]:
json1 = json.dumps('cluster_state.json', sort_keys=True)
json2 = json.dumps('cluster_state_TEST.json', sort_keys=True)
print(f'comparison: {json1 != json2}')

comparison: True


In [37]:
with open('cluster_state.json') as f1, open('cluster_state_TEST.json') as f2:
    data1 = json.load(f1)
    data2 = json.load(f2)
    something = data1 == data2
    print(something)

False


In [38]:
with open('cluster_state.json') as f1, open('cluster_state_TEST.json') as f2:
    pass

In [42]:
qqv = get_qq_version(rest_client)

In [44]:
type(qqv)

str